Необходимо реализовать метод,
который будет принимать путь до изображения и выдавать целочисленное значение - цену, распознанную на этом изображении.

Описание данных:

- выборка состоит из
изображений и цен к ним
- изображения и цены можно идентифицировать по информации, находящейся в train.csv

 Ожидание от решения:


- исполняемый скрипт на python или
jupyter notebook
- желательно оформить решение в
виде класса или функции
- нужно объяснить выбор алгоритма
для решения
- предоставить веса для модели (расчет метрик дополнительно произведется на наших данных) и результаты работы модели (распознать все изображения из test.csv и добавить в него колонку с результатами)
- целевые метрики: accuracy,
precision, recall.

На что будет обращено внимание при
проверке:

- читаемость и воспроизводимость
кода, наличие комментариев
- выбор и реализация метода,
решающего задачу
- качество метода, отсутствие
переобучения.

Результат с исходниками в виде скриптов/ноутбуков и пояснениями в виде readme.md оформить в публичный гитхаб репозиторий и ссылку вставить в поле ниже в этой форме. Хранение весов и результатов распознавания допускается в репозитории/гугл диске.

Дедлайн: 7 дней с момента получения задания. 

In [28]:
import pandas as pd
import numpy as np
import cv2
import pytesseract
import os
import re
import matplotlib.pyplot as plt
from tqdm import tqdm 
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [29]:
#Загрузим данные
IMAGE_DIR = 'imgs/'

# Загрузка обучающего набора
train_df = pd.read_csv('train.csv')

# Загрузка валидационного набора
val_df = pd.read_csv('val.csv')

In [30]:
# Посмотрим сколько значений не уникальны
duplicates = train_df['text'].value_counts()
duplicates = duplicates[duplicates > 1]

unique_prices = train_df['text'].unique()
price_series = pd.Series(unique_prices)

price_series, duplicates

(0       109.0
 1        64.0
 2       101.0
 3       229.0
 4        39.0
         ...  
 291     797.0
 292    2049.0
 293     634.0
 294     121.0
 295    4899.0
 Length: 296, dtype: float64,
 text
 149.0     127
 119.0     126
 139.0     116
 99.0      115
 89.0      111
          ... 
 3599.0      2
 1329.0      2
 8699.0      2
 1359.0      2
 3499.0      2
 Name: count, Length: 249, dtype: int64)

In [31]:
# Функция для предобработки изображения
def preprocess_image(image_path):
    """
    Загружает изображение, конвертирует его в оттенки серого и применяет бинаризацию (порог Оцу).
    Если изображение не загружено, возвращает None.
    """
    image = cv2.imread(image_path)
    if image is None:
        return None
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Бинаризация с автоматическим порогом (Оцу)
    _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    return binary

In [32]:
# Функция для извлечения цены из изображения
def extract_price_from_image(image_path):
    """
    Предобрабатывает изображение и с помощью Tesseract извлекает найденное число.
    Если числа не найдено или изображение не загружено, возвращает None.
    """
    processed_image = preprocess_image(image_path)
    if processed_image is None:
        return None
    text = pytesseract.image_to_string(processed_image, config='--psm 6')
    numbers = re.findall(r'\d+', text)
    return int(numbers[0]) if numbers else None

In [33]:
# Функция для получения предсказаний по DataFrame
def get_predictions(df, image_dir):
    """
    Для каждого изображения, указанного в столбце 'img_name' DataFrame,
    проверяет существование файла и извлекает цену.
    Результат записывается в новый столбец 'predicted_price'.
    """
    predicted_prices = []
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Обработка изображений"):
        image_path = os.path.join(image_dir, row['img_name'])
        if os.path.exists(image_path):
            price = extract_price_from_image(image_path)
            predicted_prices.append(price)
        else:
            predicted_prices.append(None)
    df['predicted_price'] = predicted_prices
    return df

In [34]:
# Функция для расчета digit-level accuracy для одного объекта
def digit_accuracy(true_value, predicted_value):
    """
    Вычисляет долю правильно предсказанных цифр между true_value и predicted_value.
    Если длины чисел различны, дополняет меньшую строку ведущими нулями до длины большей.
    Если predicted_value отсутствует (NaN), возвращает 0.
    """
    if pd.isna(predicted_value):
        return 0.0
    # Преобразуем значения в строки
    true_str = str(true_value)
    pred_str = str(predicted_value)
    # Выравниваем длины, добавляя ведущие нули
    max_len = max(len(true_str), len(pred_str))
    true_str = true_str.zfill(max_len)
    pred_str = pred_str.zfill(max_len)
    # Подсчитываем совпадающие цифры на соответствующих позициях
    correct = sum(1 for t, p in zip(true_str, pred_str) if t == p)
    return correct / max_len

In [35]:
def within_tolerance(true_value, predicted_value, tol=0.05):
    """
    Возвращает True, если относительная ошибка (|true - pred| / true) не превышает tol (например, 5%).
    Если predicted_value отсутствует, возвращает False.
    """
    if pd.isna(predicted_value):
        return False
    # Предполагаем, что истинное значение не равно 0
    return abs(true_value - predicted_value) / true_value <= tol

In [36]:
# Загрузка валидационного и тестового предсказанного набора
val_df = get_predictions(val_df, IMAGE_DIR)
train_df = get_predictions(train_df, IMAGE_DIR)

Обработка изображений: 100%|██████████| 4952/4952 [11:00<00:00,  7.50it/s]


In [37]:
# 1. Digit-Level Accuracy: вычисляем для каждой строки и усредняем
train_df['digit_accuracy'] = train_df.apply(lambda row: digit_accuracy(row['text'], row['predicted_price']), axis=1)
val_df['digit_accuracy'] = val_df.apply(lambda row: digit_accuracy(row['text'], row['predicted_price']), axis=1)
digit_level_acc_train = train_df['digit_accuracy'].mean()
digit_level_acc_val   = val_df['digit_accuracy'].mean()
print("Train Digit-Level Accuracy:", digit_level_acc_train)
print("Validation Digit-Level Accuracy:", digit_level_acc_val)

# 2. Recognition Rate: доля изображений с полученным предсказанием
recognition_rate_train = train_df['predicted_price'].notna().mean()
recognition_rate_val = val_df['predicted_price'].notna().mean()
print("Train Recognition Rate:", recognition_rate_train)
print("Validation Recognition Rate:", recognition_rate_val)

# 3. Simple Accuracy (точное совпадение чисел)
# Если предсказание отсутствует (NaN), заменяем на -1, чтобы его можно было сравнить с истинным значением.
train_df['predicted_price_filled'] = train_df['predicted_price'].fillna(-1)
val_df['predicted_price_filled'] = val_df['predicted_price'].fillna(-1)
simple_accuracy_train = (train_df['text'] == train_df['predicted_price_filled']).mean()
simple_accuracy_val = (val_df['text'] == val_df['predicted_price_filled']).mean()
print("Train Simple Accuracy (exact match):", simple_accuracy_train)
print("Validation Simple Accuracy (exact match):", simple_accuracy_val)

# 4. Custom Accuracy по допуску: считаем предсказание верным, если относительная ошибка не превышает 5%
train_df['custom_accuracy'] = train_df.apply(lambda row: 1 if within_tolerance(row['text'], row['predicted_price'], tol=0.05) else 0, axis=1)
val_df['custom_accuracy'] = val_df.apply(lambda row: 1 if within_tolerance(row['text'], row['predicted_price'], tol=0.05) else 0, axis=1)
custom_acc_train = train_df['custom_accuracy'].mean()
custom_acc_val = val_df['custom_accuracy'].mean()
print("Train Custom Accuracy (within 5% tolerance):", custom_acc_train)
print("Validation Custom Accuracy (within 5% tolerance):", custom_acc_val)

# 3. Simple Accuracy (точное совпадение чисел)
# Если предсказание отсутствует (NaN), заменяем на -1, чтобы его можно было сравнить с истинным значением.
train_df['predicted_price_filled'] = train_df['predicted_price'].fillna(-1)
val_df['predicted_price_filled'] = val_df['predicted_price'].fillna(-1)
simple_accuracy_train = (train_df['text'] == train_df['predicted_price_filled']).mean()
simple_accuracy_val = (val_df['text'] == val_df['predicted_price_filled']).mean()
print("Train Simple Accuracy (exact match):", simple_accuracy_train)
print("Validation Simple Accuracy (exact match):", simple_accuracy_val)

# 4. Стандартные метрики (Accuracy, Precision, Recall) из sklearn
y_true_train = train_df['text']
y_pred_train = train_df['predicted_price_filled']
acc_sklearn_train = accuracy_score(y_true_train, y_pred_train)
prec_sklearn_train = precision_score(y_true_train, y_pred_train, average='macro', zero_division=0)
rec_sklearn_train = recall_score(y_true_train, y_pred_train, average='macro', zero_division=0)
print("Train Accuracy (sklearn):", acc_sklearn_train)
print("Train Precision (sklearn):", prec_sklearn_train)
print("Train Recall (sklearn):", rec_sklearn_train)

y_true_val = val_df['text']
y_pred_val = val_df['predicted_price_filled']
acc_sklearn_val = accuracy_score(y_true_val, y_pred_val)
prec_sklearn_val = precision_score(y_true_val, y_pred_val, average='macro', zero_division=0)
rec_sklearn_val = recall_score(y_true_val, y_pred_val, average='macro', zero_division=0)
print("Validation Accuracy (sklearn):", acc_sklearn_val)
print("Validation Precision (sklearn):", prec_sklearn_val)
print("Validation Recall (sklearn):", rec_sklearn_val)

# 5. Custom Accuracy (предсказание считается верным, если относительная ошибка не превышает 5%)
train_df['custom_accuracy'] = train_df.apply(lambda row: 1 if within_tolerance(row['text'], row['predicted_price'], tol=0.05) else 0, axis=1)
val_df['custom_accuracy'] = val_df.apply(lambda row: 1 if within_tolerance(row['text'], row['predicted_price'], tol=0.05) else 0, axis=1)
custom_accuracy_train = train_df['custom_accuracy'].mean()
custom_accuracy_val = val_df['custom_accuracy'].mean()
print("Train Custom Accuracy (within 5% tolerance):", custom_accuracy_train)
print("Validation Custom Accuracy (within 5% tolerance):", custom_accuracy_val)

Train Digit-Level Accuracy: 0.8110936418186014
Validation Digit-Level Accuracy: 0.7994761904761903
Train Recognition Rate: 0.8649030694668821
Validation Recognition Rate: 0.861
Train Simple Accuracy (exact match): 0.720718901453958
Validation Simple Accuracy (exact match): 0.697
Train Custom Accuracy (within 5% tolerance): 0.7275848142164781
Validation Custom Accuracy (within 5% tolerance): 0.707
Train Simple Accuracy (exact match): 0.720718901453958
Validation Simple Accuracy (exact match): 0.697
Train Accuracy (sklearn): 0.720718901453958
Train Precision (sklearn): 0.5725459296459555
Train Recall (sklearn): 0.47407546773811754
Validation Accuracy (sklearn): 0.697
Validation Precision (sklearn): 0.6401529516849067
Validation Recall (sklearn): 0.5482012008779612
Train Custom Accuracy (within 5% tolerance): 0.7275848142164781
Validation Custom Accuracy (within 5% tolerance): 0.707


In [ ]:
final_df = train_df.drop(columns = ['predicted_price', 'digit_accuracy', 'custom_accuracy'])

# Создадим csv с результатами
final_df.to_csv('final_df.csv')

# Результаты
#### Digit-Level Accuracy (Train ~81%, Val ~80%)

Эта метрика показывает, что на уровне отдельных цифр модель правильно распознаёт около 80% разрядов числа. Это означает, что если цена состоит из, например, 3–4 цифр, в среднем 2,5–3 цифры будут распознаны верно. Это неплохо, особенно если учитывать, что при ошибках не обязательно все разряды будут неверными.

### Recognition Rate (Train ~86%, Val ~86%)

Эта метрика отражает долю изображений, для которых модель смогла выдать какое-либо предсказание (т.е. значение не равно NaN). При 86% таких случаев можно сказать, что в большинстве изображений система работает и возвращает результат.

### Custom Accuracy (Within 5% Tolerance) (Train ~73%, Val ~71%)

Эта метрика считает предсказание верным, если относительная ошибка не превышает 5%. Результаты почти совпадают с точным совпадением, что может говорить о том, что когда модель ошибается, ошибка либо минимальна, либо модель часто выдаёт абсолютно верный результат.

### Accuracy
(Train ~72%, Val ~70%) – Здесь мы измеряем процент случаев, когда предсказанное число полностью совпадает с истинным значением. Значения 72% на обучающей выборке и 70% на валидационной говорят о том, что примерно в 7 из 10 случаев модель предсказывает цену абсолютно точно.

### Precision
(Train ~57%, Val ~64%) 

### Recall
(Train ~45%, Val ~55%) – эти метрики, рассчитанные для задачи, где каждая цена считается отдельным классом, показывают, что при строгом критерии точного совпадения модель ошибается на некоторых классах чаще, что снижает полноту (Recall) и точность (Precision). Обычно это связано с дисбалансом классов и тем, что при небольшом количестве образцов для некоторых цен метрика оказывается ниже.  

# Общий вывод:
Метрики на обучающем и валидационном наборах очень близки, что указывает на хорошую обобщаемость модели и отсутствие переобучения.
Digit-Level Accuracy около 80% означает, что на уровне отдельных цифр система работает достаточно хорошо.
Exact match accuracy (около 70%) и custom accuracy (с допуском 5%) также показывают стабильные результаты.
Стандартные метрики (precision и recall) немного ниже, что естественно для задачи с большим количеством классов (каждая цена – отдельный класс) и могут служить дополнительным ориентиром.